In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors
from IPython.display import Image

In [ ]:
covid_vaccination = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
covid_vaccination.head(10)

In [ ]:
covid_vaccination.info()

In [ ]:
info_covid = covid_vaccination.isnull().sum()

In [ ]:
info_covid

### Task 1 - Which country is using what vaccine?

In [ ]:
covid_vaccination['country_vac'] = covid_vaccination['country'] + ' vaccine is used - ' +  covid_vaccination['vaccines']

In [ ]:
for i in covid_vaccination['country_vac'].unique():
    print(i)

### Task 2 - In which country the vaccination programme is more advanced?

In [ ]:
countries_list = covid_vaccination.country.unique()
print(countries_list)

In [ ]:
vaccine_df = covid_vaccination[covid_vaccination['country'].isin(countries_list)].fillna(0)

In [ ]:
vaccine_df.head(100)

In [ ]:
percent_vacc = vaccine_df.groupby(['country'], sort=False, as_index=False)['people_vaccinated_per_hundred'] \
.max() \
.sort_values(by='people_vaccinated_per_hundred', ascending=False)
percent_vacc

In [ ]:
percent_fvacc = vaccine_df.groupby(['country'], sort=False, as_index=False)['people_fully_vaccinated_per_hundred'] \
.max() \
.sort_values(by='people_fully_vaccinated_per_hundred', ascending=False)
percent_fvacc

In [ ]:
precent_vacc_f_vacc = percent_vacc.merge(percent_fvacc, how='outer', on='country').fillna(0)
precent_vacc_f_vacc

In [ ]:
percent_vacc = precent_vacc_f_vacc.groupby(['country'], sort=False)['people_vaccinated_per_hundred'].max()
percent_vacc

In [ ]:
percent_fvacc = precent_vacc_f_vacc.groupby(['country'], sort=False)['people_fully_vaccinated_per_hundred'].max()
percent_fvacc

In [ ]:
ax = percent_vacc.iloc[:15].plot(kind='bar', figsize=(10,6), color="royalblue", fontsize=13,);
ax = percent_fvacc.iloc[:15].plot(kind='bar', figsize=(10,6), color="orange", fontsize=13,);

ax.grid(which='major',zorder=1, alpha=0.5)
ax.grid(which='minor', alpha=1)

ax.legend()
ax.set_alpha(0.2)
ax.set_title("15 Most popular countries for vaccination", fontsize=22)
ax.set_ylabel("Percent", fontsize=15)
ax.set_xlabel("Country", fontsize=15);
plt.show()

### Task 3 - Where are vaccinated more people per day? But in terms of percent from entire population ?

In [ ]:
max_covid_day = covid_vaccination.fillna(0).groupby(['country'], sort=False, as_index=False) \
.agg({'daily_vaccinations': 'max'})

In [ ]:
max_covid_day

In [ ]:
days_covid_country = covid_vaccination.fillna(0).merge(max_covid_day, how='inner', on=['country', 'daily_vaccinations'])

In [ ]:
days_covid_country

In [ ]:
fig = px.bar(
  days_covid_country.sort_values('daily_vaccinations'), 
  x='date', y='daily_vaccinations',
  color='country',
)
fig.update_layout(
  template='plotly_dark',
  margin=dict(r=10, t=25, b=40, l=60)
)
fig.show()

In [ ]:
country_covid = days_covid_country.query(f'daily_vaccinations == {days_covid_country.daily_vaccinations.max()}')
country_covid

In [ ]:
print(f'The maximum percentage of vaccinated in the whole country: {country_covid["people_fully_vaccinated_per_hundred"].iloc[0]}')